# **Performance improvement using Mixed Precision**

In this implementation, we will be observing the performance improvement we get in the implementation of CNN by using the concept of mixed precicion. The dataset is the "Chest X-ray Images" data. It contains two classes, thus it is a binary classification problem. 

In [ ]:
#importing libraries
import pandas as pd
import numpy as np

import cv2
from google.colab.patches import cv2_imshow

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import zipfile
with zipfile.ZipFile("/content/drive/My Drive/chest_xray.zip",'r') as f:
  f.extractall(' ')

Here we have to select the "Tesla T4" Gpu, to observe the performance increase. 

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-1d9eab4a-3230-18b4-c22d-dec896b848c6)


The mixed precision policy is set to "mixed_float16" 

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

In [ ]:
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Compute dtype: float16
Variable dtype: float32


In [ ]:
import os
image_names = os.listdir("/content/ /chest_xray/train/NORMAL")

In [ ]:
len(image_names)

1342

In [ ]:
x_train = []
y_train = []
for i in range(len(image_names)):
  try:
    img = cv2.imread("/content/ /chest_xray/train/NORMAL/" + image_names[i])
    img = cv2.resize(img,(64,64))
    x_train.append(img)
    y_train.append(0)
  except:
    pass
  
len(x_train)

1341

In [ ]:
image_names = os.listdir("/content/ /chest_xray/train/PNEUMONIA")


In [ ]:
for i in range(len(image_names)):
  try:
    img = cv2.imread("/content/ /chest_xray/train/PNEUMONIA/" + image_names[i])
    img = cv2.resize(img,(64,64))
    x_train.append(img)
    y_train.append(1)
  except:
    pass

In [ ]:
len(x_train)

5216

In [ ]:
image_names = os.listdir("/content/ /chest_xray/test/NORMAL")
x_test = []
y_test = []
for i in range(len(image_names)):
  try:
    img = cv2.imread("/content/ /chest_xray/test/NORMAL/" + image_names[i])
    img = cv2.resize(img,(64,64))
    x_test.append(img)
    y_test.append(0)
  except:
    pass

In [ ]:
len(x_test)

234

In [ ]:
image_names = os.listdir("/content/ /chest_xray/test/PNEUMONIA")
for i in range(len(image_names)):
  try:
    img = cv2.imread("/content/ /chest_xray/test/PNEUMONIA/" + image_names[i])
    img = cv2.resize(img,(64,64))
    x_test.append(img)
    y_test.append(1)
  except:
    pass

In [ ]:
len(x_test), len(y_test)

(624, 624)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
import numpy as np

In [ ]:
x_train[0].shape


(64, 64, 3)

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
np.unique(y_train)


array([0, 1])

In [ ]:
x_train.shape


(5216, 64, 64, 3)

In the below cell, the CNN is implemented. While using the mixed precision policy for observing the performance improvement, some conditions must be followed. These conditions are:

1) The number of filters should be a multiple of 8.

2) The number of neurons in the fully connected layer should also a multiple of 8.

In [ ]:
model = Sequential()
model.add(Conv2D(48,(3,3),activation='relu',input_shape=(64,64,3)))
model.add(MaxPooling2D())
model.add(Conv2D(48,(3,3),activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=10,batch_size=128)

Epoch 1/10
41/41 [==============================] - 1s 16ms/step - loss: 3.7034 - accuracy: 0.7048
Epoch 2/10
41/41 [==============================] - 1s 13ms/step - loss: 0.1344 - accuracy: 0.9475
Epoch 3/10
41/41 [==============================] - 1s 13ms/step - loss: 0.0955 - accuracy: 0.9641
Epoch 4/10
41/41 [==============================] - 1s 13ms/step - loss: 0.0733 - accuracy: 0.9718
Epoch 5/10
41/41 [==============================] - 1s 13ms/step - loss: 0.0787 - accuracy: 0.9712
Epoch 6/10
41/41 [==============================] - 1s 13ms/step - loss: 0.0625 - accuracy: 0.9757
Epoch 7/10
41/41 [==============================] - 1s 13ms/step - loss: 0.0580 - accuracy: 0.9799
Epoch 8/10
41/41 [==============================] - 1s 13ms/step - loss: 0.0499 - accuracy: 0.9812
Epoch 9/10
41/41 [==============================] - 1s 13ms/step - loss: 0.0393 - accuracy: 0.9845
Epoch 10/10
41/41 [==============================] - 1s 13ms/step - loss: 0.0288 - accuracy: 0.9891


In [ ]:
x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
model.evaluate(x_test, y_test)

20/20 [==============================] - 0s 3ms/step - loss: 1.4493 - accuracy: 0.7436


[1.4493356943130493, 0.7435897588729858]

**Note:**

The increase in performance will be more significant in a larger datdset as compared to a smaller datadset. 
